# Estudo da composição das previsões do rating final

O objetivo é realizar previsões para toda base gerando um rating unindo o contribuinte e posteriormente o igr gerado na regressão

In [1]:
import os
import dotenv
import boto3
from io import BytesIO
import pandas as pd
import numpy as np
from datetime import date
import zipfile
from sklearn.preprocessing import StandardScaler, RobustScaler
import pickle

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

import warnings

warnings.filterwarnings("ignore")

print("Processo iniciado")

# Define diretorios
rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

def formatar_moeda(valor):
    return locale.currency(valor, grouping=True)

Processo iniciado


In [ ]:
# print("Carregando dados para previsões")

# # Prepara os dados para realizar as previsões
# zip_file = os.path.join(dataPath, 'estoque_da3.zip') # 14bi
# z = zipfile.ZipFile(zip_file)


# def ler_bases_exportadas(nome_arquivo):
#     z.extract(nome_arquivo)
#     df = pd.read_csv(nome_arquivo, sep=',')
#     os.remove(nome_arquivo)
#     return df


# base_conjunta = ler_bases_exportadas('estoque-divida.csv')

In [2]:
s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACESS_KEY")
    )

def read_s3(bucket_name, folder_name, file_name):
    file_key_aws = folder_name + file_name
    obj = s3.Bucket(bucket_name).Object(file_key_aws).get()
    df = pd.read_csv(obj['Body'], sep=';')
    return df

In [3]:
print("Carregando dados para previsões")

dados_predict = read_s3(
     bucket_name=os.getenv("S3_BUCKET_NAME"), 
     folder_name=os.getenv("S3_FOLDER_NAME"), 
     file_name='feature_store_igr_prime_ab1_maxpg_lari.csv')

Carregando dados para previsões


In [4]:
dados_contribuinte = read_s3(
    bucket_name = os.getenv("S3_BUCKET_NAME"), 
    folder_name = os.getenv("S3_FOLDER_NAME"), 
    file_name = 'feature_store_contribuinte_prime_k4_maxpg_lari.csv')

In [6]:
formatar_moeda(dados_predict['valor_aberto'].sum())

'R$ 13.863.282.107,90'

In [5]:
dados_predict.shape[0]

967079

In [6]:
# Renomeia colunas para nome mais adequados e filtra dataframe
colunas_nome = {
    'valor_tot': 'valor_total_da',
}
dados_predict = dados_predict.rename(columns=colunas_nome)

In [7]:
# Filtrando variáveis de interesse para a modelagem
df_divida = dados_predict[['cda', 
                          'tipo_divida',
                          'id_pessoa', 
                          'da_aberto',
                          'percentual_pago_cda', 
                          'valor_total_da', 
                          'idade_divida'#, 
                          #'quantidade_reparcelamento' 
                          ]] 

In [8]:
dados_contribuinte_da_ab_1 = dados_contribuinte[dados_contribuinte['da_aberto'] == 1]

In [9]:
dados_contribuinte_da_ab_1.shape[0]

279212

In [10]:
df_contribuinte = dados_contribuinte_da_ab_1[['id_pessoa', 
                                      'tipo_divida',
                                      'num_dist_cda', 
                                      'quantidade_reparcelamento',
                                      #'historico_pagamento_em_qtd',
                                       'historico_pagamento_em_valor',
                                       'situacao_cobranca',
                                       'class_contribuinte',
                                       'class_contribuinte_peso']]

In [11]:
df = pd.merge(df_divida, df_contribuinte, on=['id_pessoa', 'tipo_divida'], how = "left")
df.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'da_aberto', 'percentual_pago_cda',
       'valor_total_da', 'idade_divida', 'num_dist_cda',
       'quantidade_reparcelamento', 'historico_pagamento_em_valor',
       'situacao_cobranca', 'class_contribuinte', 'class_contribuinte_peso'],
      dtype='object')

In [12]:
# Métricas p acompanhar
num_cda_01 = df['cda'].nunique() # 967.079
num_pessoa_01 = df['id_pessoa'].nunique() # 255.253
vlr_da_01 = df['valor_total_da'].sum() # 'R$ 13.863.337.109,58'

In [15]:
num_cda_01

967079

In [13]:
num_pessoa_01

255253

In [14]:
formatar_moeda(vlr_da_01)

'R$ 13.863.337.109,58'

In [15]:
df.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'da_aberto', 'percentual_pago_cda',
       'valor_total_da', 'idade_divida', 'num_dist_cda',
       'quantidade_reparcelamento', 'historico_pagamento_em_valor',
       'situacao_cobranca', 'class_contribuinte', 'class_contribuinte_peso'],
      dtype='object')

In [16]:
print("Preparação do pipeline de previsões")
# Filtra os dados que precisamos para previsão
df_feature_store = df[['valor_total_da', 
                       'idade_divida', #'anos_idade_da',
                        'quantidade_reparcelamento',
                       'num_dist_cda', #'frequencia_da_pessoa',
                        #'historico_pagamento_em_qtd', 
                        'situacao_cobranca', #'status_situacao',
                       'historico_pagamento_em_valor', 
                       'class_contribuinte_peso']]
df_feature_store.shape[0]


Preparação do pipeline de previsões


967079

In [17]:
# Normaliza os dados para previsão utilizando o StandardScaler
normalizador = StandardScaler() # RobustScaler()
normalizador.fit(df_feature_store)
dados_normalizados = normalizador.fit_transform(df_feature_store)

colunas = list(normalizador.get_feature_names_out())
df_normalizado = pd.DataFrame(dados_normalizados, columns=colunas)

In [18]:
df_normalizado.shape[0]

967079

In [19]:
print("Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)")


def abre_modelo(nome_modelo, path_modelo, zip_name=None):
    if zip_name:
        zip_file = os.path.join(path_modelo, zip_name)
        z = zipfile.ZipFile(zip_file)
        z.extract(nome_modelo)
    else:
        nome_modelo = os.path.join(path_modelo, nome_modelo)

    modelo = pickle.load(open(nome_modelo, 'rb'))
    return modelo


model_predict_igr = abre_modelo("modeloDA-igr-divida-prime_maxpg_lari.pkl", modelsPath)

Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)


In [20]:
# Realizando previsões
previsoes = model_predict_igr.predict(df_normalizado)
df['igr'] = previsoes

In [22]:
df[df['cda'] == '1491939ceb02c7cfe52281a70553db462']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr
77158,1491939ceb02c7cfe52281a70553db462,mercantil,2f48e5117fc31832,1,0.0,3298.56,1.0,1,0,0.0,0.0,4,0.10483,0.0


In [23]:
df.loc[df['situacao_cobranca'] == 0, 'igr'] = 0

In [24]:
df.loc[df['idade_divida'] >= 15, 'igr'] = 0

In [25]:
df.shape[0]

967079

In [26]:
df[df['cda'] == '1491939ceb02c7cfe52281a70553db462']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr
77158,1491939ceb02c7cfe52281a70553db462,mercantil,2f48e5117fc31832,1,0.0,3298.56,1.0,1,0,0.0,0.0,4,0.10483,0.0


In [26]:
dados_contribuinte.columns

Index(['id_pessoa', 'tipo_divida', 'num_dist_cda', 'quantidade_reparcelamento',
       'valor_tot', 'valor_pago', 'qtd_notas_2anos', 'edificacao', 'situacao',
       'cpf_cnpj_existe', 'endereco_existe', 'perfil_acessivel',
       'situacao_cobranca', 'historico_pagamento_em_valor',
       'class_situacao_cobranca', 'class_contribuinte', 'da_aberto',
       'class_contribuinte_nome', 'class_contribuinte_perfil',
       'percentual_pago_cda', 'class_contribuinte_peso'],
      dtype='object')

In [27]:
# #dados_contribuinte.loc[dados_contribuinte['class_contribuinte'] == 4, ['class_contribuinte', 'class_contribuinte_nome', 'class_contribuinte_perfil']]
# contrib = dados_contribuinte.groupby(['class_contribuinte', 'class_contribuinte_nome', 'class_contribuinte_perfil'])['id_pessoa'].nunique().to_frame().reset_index()

# dict = {"class_contribuinte_nome": ['PRIMEIRA DIVIDA', 'DEVEDOR EXCEL', 
#                                     'DEVEDOR BOM', 'DEVEDOR RUIM', 'DEVEDOR PESSIMO'],
#         "ordem_contrib": [0, 1, 2 ,3, 4]}

# df_ord_class_contrib = pd.DataFrame(dict)

# contrib = pd.merge(contrib, df_ord_class_contrib, on = 'class_contribuinte_nome').sort_values(by = "ordem_contrib")
# total = contrib['id_pessoa'].sum()
# contrib['perc'] = np.round(contrib['id_pessoa']/total, 5)
# contrib

,class_contribuinte,class_contribuinte_nome,class_contribuinte_perfil,id_pessoa,ordem_contrib,perc
4,4,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,161663,0,0.29868
2,2,DEVEDOR EXCEL,PAGA NAO REPARC,128380,1,0.23719
1,1,DEVEDOR BOM,PAGA REPARC,33664,2,0.06220
3,3,DEVEDOR RUIM,REPARCELADOR,3922,3,0.00725
0,0,DEVEDOR PESSIMO,NAO PAGA,213633,4,0.39469


# Rótulos da Dívida

In [29]:
def make_rating_divida_original(dataframe):

    dataframe.loc[dataframe['igr'] == 0, 'rating_divida'] = 'BAIXISSIMA'

    # Devedor Excelente (PAGA NAO REPARC)
    dataframe.loc[(dataframe['rating_divida'].isnull()) & (dataframe['class_contribuinte'] == 2), 'rating_divida'] = 'ALTISSIMA'

    # Devedor bom (PAGA REPARC)
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.1), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    # Devedor ruim (REPARCELADOR)
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.05), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.05) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # Devedor Péssimo (NAO PAGA)
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    ## PRIMEIRA DIVIDA (class_contribuinte == 4)
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] >= 0.3), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] >= 0.1) & (dataframe['igr'] < 0.3) , 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] != 0) & (dataframe['igr'] < 0.1) , 'rating_divida'] = 'MEDIA'

# dicionario_clusteres = {
#     'class_contribuinte': [0, 1, 2, 3, 4],
#     'class_contribuinte_nome': ['DEVEDOR PESSIMO',
#                                 'DEVEDOR BOM',
#                                 'DEVEDOR EXCEL',
#                                 'DEVEDOR RUIM',
#                                 'PRIMEIRA DIVIDA'],
 
#     'class_contribuinte_perfil': ['NAO PAGA',
#                                 'PAGA REPARCELADO',
#                                 'PAGA NAO REPARCELA',
#                                 'REPARCELADOR',
#                                 'NOVO EM DIVIDA']
# }
 
# df_dicionario_clusteres = pd.DataFrame(dicionario_clusteres)
# df_dicionario_clusteres


    return dataframe


### Outra abordagem

In [33]:
def make_rating_divida_quantil(dataframe):
    
    dataframe.loc[dataframe['igr'] == 0, 'rating_divida'] = 'BAIXISSIMA'

    # Devedor Excelente (PAGA NAO REPARC)
    dataframe.loc[(dataframe['rating_divida'].isnull()) & (dataframe['class_contribuinte'] == 2), 'rating_divida'] = 'ALTISSIMA'


    # Devedor bom (PAGA REPARC)
    q075_dev_bom = dataframe.loc[(dataframe['class_contribuinte'] == 1), 'igr'].quantile(0.75)
    q050_dev_bom = dataframe.loc[(dataframe['class_contribuinte'] == 1), 'igr'].quantile(0.5)

    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] >= q075_dev_bom), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < q075_dev_bom) & (dataframe['igr'] >= q050_dev_bom), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < q050_dev_bom) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'


    # Devedor ruim (REPARCELADOR)
    q075_dev_ruim = dataframe.loc[(dataframe['class_contribuinte'] == 3), 'igr'].quantile(0.75)
    q050_dev_ruim = dataframe.loc[(dataframe['class_contribuinte'] == 3), 'igr'].quantile(0.5)

    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= q075_dev_ruim), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < q075_dev_ruim) & (dataframe['igr'] >= q050_dev_ruim), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < q050_dev_ruim) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'


    # Devedor Péssimo (NAO PAGA)
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'


    ## PRIMEIRA DIVIDA (class_contribuinte == 4)
    dataframe.loc[(dataframe['class_contribuinte'] == 4), 'rating_divida'] = 'ALTA'


    return dataframe

In [55]:
print("Classificação do rating da dívida parametrizando junto a classificação do contribuinte - método ORIGINAL")
df_original = make_rating_divida_original(df)
df_original

Classificação do rating da dívida parametrizando junto a classificação do contribuinte - método ORIGINAL


,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,1,0.0,2048.73,1.0,2,0,0.0,4.0,2,1.56033,0.670109,ALTISSIMA
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,1,0.0,2515.85,1.0,3,0,0.0,0.0,0,-1.27335,0.000000,BAIXISSIMA
2,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,1,0.0,847.02,0.0,4,3,0.0,0.0,0,-1.27335,0.000000,BAIXISSIMA
3,0000331f601a73e52b46f673bf0c61252,mercantil,e8424494daac9641,1,0.0,5385.81,11.0,4,0,0.0,0.0,0,-1.27335,0.000000,BAIXISSIMA
4,00003d46e618da886b3a8e268c80aedf1,imovel,d8b23eda9800b9e3,1,0.0,6434.96,12.0,4,0,0.0,0.0,0,-1.27335,0.000000,BAIXISSIMA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967074,ffffca43e897bbf0d8a3e5cebc255ca61,imovel,cad70f5b8701af03,1,0.0,5082.97,11.0,5,0,0.0,0.0,0,-1.27335,0.000000,BAIXISSIMA
967075,ffffca43e897bbf0d8a3e5cebc255ca62,mercantil,e750ab1d88feb1ba,1,0.0,7634.36,11.0,6,0,0.0,0.0,0,-1.27335,0.000000,BAIXISSIMA
967076,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,1,0.0,1080.38,1.0,3,2,0.0,0.0,0,-1.27335,0.000000,BAIXISSIMA
967077,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,1,0.0,9390.43,25.0,5,0,0.0,4.0,2,1.56033,0.000000,BAIXISSIMA


In [56]:
df_original[df_original['cda'] == '1491939ceb02c7cfe52281a70553db462']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
77158,1491939ceb02c7cfe52281a70553db462,mercantil,2f48e5117fc31832,1,0.0,3298.56,1.0,1,0,0.0,0.0,4,0.10483,0.0,BAIXISSIMA


In [57]:
# print("Classificação do rating da dívida parametrizando junto a classificação do contribuinte - método QUANTIL")
# df_quantil = make_rating_divida_quantil(df)
# df_quantil

### Comparar versão original com quantil 

In [35]:
# df_orig_comparar = df_original[['cda', 'tipo_divida', 'id_pessoa', 'rating_divida']]
# df_orig_comparar = df_orig_comparar.rename(columns = {'rating_divida':'rating_divida_orig'})

In [36]:
# df_quantil_comparar = df_quantil[['cda', 'tipo_divida', 'id_pessoa', 'rating_divida']]
# df_quantil_comparar = df_quantil_comparar.rename(columns = {'rating_divida':'rating_divida_quantil'})

In [37]:
# comparar = pd.merge(df_orig_comparar, df_quantil_comparar, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'outer')

# rotulos_diferentes = comparar[comparar['rating_divida_orig'] != comparar['rating_divida_quantil']]

In [38]:
# comparar#[['rating_divida_orig', 'rating_divida_quantil']].value_counts().to_frame().reset_index().sort_values(by = ['rating_divida_orig', 'rating_divida_quantil'])

,cda,tipo_divida,id_pessoa,rating_divida_orig,rating_divida_quantil
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,ALTISSIMA,ALTISSIMA
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,BAIXISSIMA,BAIXISSIMA
2,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,BAIXISSIMA,BAIXISSIMA
3,0000331f601a73e52b46f673bf0c61252,mercantil,e8424494daac9641,BAIXISSIMA,BAIXISSIMA
4,00003d46e618da886b3a8e268c80aedf1,imovel,d8b23eda9800b9e3,BAIXISSIMA,BAIXISSIMA
...,...,...,...,...,...
967074,ffffca43e897bbf0d8a3e5cebc255ca61,imovel,cad70f5b8701af03,BAIXISSIMA,BAIXISSIMA
967075,ffffca43e897bbf0d8a3e5cebc255ca62,mercantil,e750ab1d88feb1ba,BAIXISSIMA,BAIXISSIMA
967076,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,BAIXISSIMA,BAIXISSIMA
967077,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,BAIXISSIMA,BAIXISSIMA


In [58]:
# contingency_table = pd.crosstab(comparar['rating_divida_orig'], comparar['rating_divida_quantil'])
# contingency_table

# Métricas

In [59]:
dict = {"rating_divida": ['ALTISSIMA', 'ALTA', 'MEDIA', 'BAIXA', 'BAIXISSIMA'],
        "ordem_rating": [0, 1, 2, 3, 4]}

df_ordenar_rating_divida = pd.DataFrame(dict)
df_ordenar_rating_divida

,rating_divida,ordem_rating
0,ALTISSIMA,0
1,ALTA,1
2,MEDIA,2
3,BAIXA,3
4,BAIXISSIMA,4


In [60]:
def numcdas_por_ratingdivida(DF, df_ordenar_rating_divida):

    # Num CDAs
    df_cdasabertas_ratdiv = DF.groupby('rating_divida')['cda'].nunique().to_frame().reset_index()
    total_cda = df_cdasabertas_ratdiv['cda'].sum()
    df_cdasabertas_ratdiv['total_cda'] = total_cda
    df_cdasabertas_ratdiv['perc_cda'] = np.round(df_cdasabertas_ratdiv['cda']/total_cda, 4)

    df_cdasabertas_ratdiv = pd.merge(df_cdasabertas_ratdiv, df_ordenar_rating_divida, on = 'rating_divida').sort_values(by = "ordem_rating")

    return df_cdasabertas_ratdiv

In [61]:
def vlrmonet_por_ratingdivida(DF, df_ordenar_rating_divida):

    # Valor monetário
    estoque_rat_div = DF.groupby('rating_divida')['valor_total_da'].sum().to_frame().reset_index()
    total_vlr_tot_da = estoque_rat_div['valor_total_da'].sum()
    estoque_rat_div['total_vlr_tot_da'] = total_vlr_tot_da
    estoque_rat_div['perc_vlr_tot_da'] = np.round(estoque_rat_div['valor_total_da']/total_vlr_tot_da, 4)

    estoque_rat_div['valor_total_da_format'] = estoque_rat_div['valor_total_da'].apply(formatar_moeda, 0)
    estoque_rat_div['total_vlr_tot_da_format'] = estoque_rat_div['total_vlr_tot_da'].apply(formatar_moeda, 0)

    estoque_rat_div = pd.merge(estoque_rat_div, df_ordenar_rating_divida, on = 'rating_divida').sort_values(by = "ordem_rating")

    return estoque_rat_div

### Original

In [62]:
df_orig_da1 = df_original[df_original['da_aberto'] == 1]

In [63]:
numcdas_por_ratingdivida(df_orig_da1, df_ordenar_rating_divida)

,rating_divida,cda,total_cda,perc_cda,ordem_rating
1,ALTISSIMA,328276,967079,0.3395,0
0,ALTA,3684,967079,0.0038,1
4,MEDIA,1145,967079,0.0012,2
2,BAIXA,69225,967079,0.0716,3
3,BAIXISSIMA,564749,967079,0.5840,4


In [64]:
vlrmonet_por_ratingdivida(df_orig_da1, df_ordenar_rating_divida)

,rating_divida,valor_total_da,total_vlr_tot_da,perc_vlr_tot_da,valor_total_da_format,total_vlr_tot_da_format,ordem_rating
1,ALTISSIMA,2.657912e+09,1.386334e+10,0.1917,"R$ 2.657.911.904,05","R$ 13.863.337.109,58",0
0,ALTA,3.680595e+07,1.386334e+10,0.0027,"R$ 36.805.954,85","R$ 13.863.337.109,58",1
4,MEDIA,6.186265e+06,1.386334e+10,0.0004,"R$ 6.186.265,44","R$ 13.863.337.109,58",2
2,BAIXA,7.361907e+08,1.386334e+10,0.0531,"R$ 736.190.683,17","R$ 13.863.337.109,58",3
3,BAIXISSIMA,1.042624e+10,1.386334e+10,0.7521,"R$ 10.426.242.302,07","R$ 13.863.337.109,58",4


### Quantil

In [46]:
# df_quant_da1 = df_quantil[df_quantil['da_aberto'] == 1]

In [65]:
# numcdas_por_ratingdivida(df_quant_da1, df_ordenar_rating_divida)

In [66]:
# vlrmonet_por_ratingdivida(df_quant_da1, df_ordenar_rating_divida)

### Métricas

In [67]:
metricas_predict = df.groupby('rating_divida')['cda'].nunique().to_frame().reset_index()
total = metricas_predict['cda'].sum()
metricas_predict['perc'] = np.round(metricas_predict['cda']/total, 5)
metricas_predict

,rating_divida,cda,perc
0,ALTA,3684,0.00381
1,ALTISSIMA,328276,0.33945
2,BAIXA,69225,0.07158
3,BAIXISSIMA,564749,0.58397
4,MEDIA,1145,0.00118


In [68]:
df_orig_da1[df_orig_da1['cda'] == '1491939ceb02c7cfe52281a70553db462']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
77158,1491939ceb02c7cfe52281a70553db462,mercantil,2f48e5117fc31832,1,0.0,3298.56,1.0,1,0,0.0,0.0,4,0.10483,0.0,BAIXISSIMA


In [72]:
df_orig_da1.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'da_aberto', 'percentual_pago_cda',
       'valor_total_da', 'idade_divida', 'num_dist_cda',
       'quantidade_reparcelamento', 'historico_pagamento_em_valor',
       'situacao_cobranca', 'class_contribuinte', 'class_contribuinte_peso',
       'igr', 'rating_divida'],
      dtype='object')

In [73]:
df_original[df_original['cda'] == '14a8fa60925e446f9fcacff532cf50c42']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
77522,14a8fa60925e446f9fcacff532cf50c42,mercantil,a6b1c07dd9a81e87,1,0.0,3298.56,1.0,1,0,0.0,0.0,4,0.10483,0.0,BAIXISSIMA


In [74]:
print("Inicia a conexão com S3 para inscrição dos dados com as previsões")

# Cria conexão ao s3 e preenche a tabela com os dados

def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())


up_s3_files(dataframe = df_original,
            bucket_name = os.getenv("S3_BUCKET_NAME"),
            folder_name = os.getenv("S3_FOLDER_NAME"),
            file_name = 'prime_maxpg_lari.csv')

print("Upload de dados efetuados no s3")
print("Processo finalizado")
print("Arquivo disponível para download e análise")

Inicia a conexão com S3 para inscrição dos dados com as previsões
Upload de dados efetuados no s3
Processo finalizado
Arquivo disponível para download e análise
